In [1]:
# 토크나이징
# soynlp와 mecab을 활용하자
###############################
# 데이터 읽어오기
import pandas as pd
df = pd.read_csv("3_spaced_text.csv", encoding="utf-8")
df.tail(3)

,search_keyword,date_created,time_created,writer,is_reply,id,spaced_text
1152309,휴젤,2020-04-29,11:55:10,ㅇㅇ(123.215),0,241400125127,미국주식보다 한국꺼보면 ㄹㅇ 암걸릴 것 같음 특히 휴젤 저거 아는분이 임원진인데 회...
1152310,휴젤,2020-04-29,11:56:16,ㅇㄷ(175.223),1,241400125127,한국 상장회사들중에 ㅂㅅ은곳이 넘 많음...
1152311,휴젤,2020-04-29,11:56:38,ㅇㅇ(223.33),1,241400125127,신흥국 회사 투자 망설여지는 게 이거 때문임. 주식시장에 대한 개념이 안 잡혀있음.


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152312 entries, 0 to 1152311
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   search_keyword  1152312 non-null  object
 1   date_created    1152312 non-null  object
 2   time_created    1152312 non-null  object
 3   writer          1152312 non-null  object
 4   is_reply        1152312 non-null  int64 
 5   id              1152312 non-null  int64 
 6   spaced_text     1152312 non-null  object
dtypes: int64(2), object(5)
memory usage: 61.5+ MB
